In [ ]:
import os
import matplotlib.pyplot as plt
import tensorflow as tf
from bayes_opt import BayesianOptimization

import numpy as np
import pandas as pd

import lib.Metrics as Metrics
from lib.models.IRNN_Full_Bayes import IRNN_Full_Bayes
from lib.models.IRNN import IRNN
from lib.train_functions import fit
from lib.utils import *
from lib.regional_data_builder import DataConstructor

print(tf.config.list_physical_devices('GPU'))

In [12]:
# Class to evaluate a set of hyper parameters. 
# Using a class allows the __call__ function to be used for different models with different configurations
# class is used wih bayesian optimization to find the best hyper parameters
class Eval_Fn:
    def __init__(self, root='', model=None, region='US', n_regions=1, n_folds = 5, test_season = 2017, gamma=28, plot=True, verbose=True, min_score=-25, batch_size=32, window_size = 28, **kwargs):
        self.model = model  
        self.n_folds = n_folds      # number of folda (k fold cross validation)
        self.test_season = test_season
        self.gamma = gamma      
        self.plot = plot            # save plots validation set forecasts  
        self.verbose = verbose      # print during training
        self.min_score = min_score  # score to give hyper paremeters if they break and get -infinity
        self.root = root            # save directory
        self.region = region
        self.window_size = window_size
        self.batch_size = batch_size
        self.n_regions = n_regions

    def __call__(self, batch_size = 32, **kwargs):
        tf.keras.backend.clear_session()
        score = {}
        plt.clf()

        kwargs['n_op'] = int(kwargs['n_op'])

        _data = DataConstructor(test_season=self.test_season, region = self.region, window_size=self.window_size, n_queries=kwargs['n_op']-1, gamma=28)
        x_train, y_train, x_test, y_test, scaler = _data()

        try:
            scaler = scaler[np.newaxis, np.newaxis, :]
        except:
            scaler = scaler.iloc[0]

        x_train = tf.cast(x_train, tf.float32)
        y_train = tf.cast(y_train, tf.float32)
        x_test = tf.cast(x_test, tf.float32)
        y_test = tf.cast(y_test, tf.float32)

        for fold in range(self.n_folds):
            try:
    
                x_val = x_train[-(365*(fold+1)): -(365*(fold)+1)]
                y_val = y_train[-(365*(fold+1)): -(365*(fold)+1)]
                val_dates = _data.train_dates[-365*(fold+1): -(365*fold)-1]

                x_tr = x_train[:-(365*(fold+1))]
                y_tr = y_train[:-(365*(fold+1))]

                train_dataset = tf.data.Dataset.from_tensor_slices((x_tr, y_tr))
                train_dataset = train_dataset.shuffle(buffer_size=1024).batch(batch_size)

                _model = self.model(**kwargs)

                # define loss, epochs learning rate
                if _model.loss == 'NLL':
                    def loss(y, p_y):
                        return -p_y.log_prob(y)
                if _model.loss == 'MSE':
                    loss = tf.keras.losses.mean_squared_error

                if 'epochs' in kwargs:
                    epochs = int(kwargs['epochs'])
                else:
                    epochs = self.epochs

                if 'lr_power' in kwargs:
                    lr = np.power(10, kwargs['lr_power'])
                else:
                    lr = 1e-3

                def loss_fn(y, p_y):
                    return -p_y.log_prob(y)

                _model(x_val)

                pred = _model(x_test)
                optimizer = tf.optimizers.Adam(learning_rate=lr)
    
                _model.compile(optimizer=optimizer, loss=loss_fn)
                _model.fit(train_dataset, epochs =epochs, verbose=self.verbose)

                y_pred = _model.predict(x_val, 25, verbose=self.verbose)

                std = (y_pred[0]+y_pred[1])[..., -self.n_regions:] * scaler - y_pred[0][..., -self.n_regions:] * scaler
                mean = y_pred[0][..., -self.n_regions:] * scaler
                y_te = y_val[..., -self.n_regions:]*scaler

                df = {g:pd.DataFrame(columns = ['True', 'Pred', 'Std'], data = np.asarray([y_te[:, g, -1], mean[:, g, -1], std[:, g, -1]]).T) for g in [6,13,20,27]}

                # get score for fold, 2 options depending on whether the forecast is a list (IRNN) or a single prediction
                try:
                    score[fold] = Metrics.nll(df[self.gamma])
                except:
                    score[fold] = np.sum(np.asarray([Metrics.nll(d) for d in df.values()]))

                # can be useful to plot the validation curves to check things are working 
                if self.plot:
                    for idx, d in enumerate(df.values()):
                        plt.subplot(len(df.keys()), 1, idx+1)
                        plt.plot(d.index, d['True'], color='black')
                        plt.plot(d.index, d['Pred'], color='red')
                        plt.fill_between(d.index, d['Pred']+d['Std'], d['Pred']-d['Std'], color='red', alpha=0.3, linewidth=0)
            except Exception as e:
                score[fold] = -self.min_score
                print(e)

        if self.plot:
            if not os.path.exists(self.root):
                os.makedirs(self.root)

            figs = os.listdir(self.root)
            nums = [-1]

            for f in figs:
                if 'fig' in f:
                    nums.append(int(f.split('_')[1].split('.')[0]))

            plt.savefig(os.path.join(self.root, 'fig_{}.pdf'.format(max(nums) + 1)))

        try:
            # NLL can be give nan values, try to prevent this breaking things
            if np.isfinite(-sum(score.values())):
                return -sum(score.values())
            else:
                return self.min_score
        except:
            return self.min_score

In [13]:
eval = Eval_Fn(model=IRNN, n_folds = 2, plot=False, verbose=False, root='Optimisation/Plots/')
test_eval_score = eval(gamma = 28,
                        epochs= 3,
                        kl_power= -2.857091693154802,
                        lr_power= -3.7364141761644545,
                        n_op= 93,
                        op_scale_pwr= -0.27139484469983133,
                        p_scale_pwr= -1.827071638197097,
                        q_scale_pwr= -1.2461978663286395,
                        rnn_units= 108
                        )

In [ ]:


max_iter =250
model = IRNN
gamma = 28
root = 'Optimisation/IRNN/'

n_folds = 4 # increase this to improve rubustness, will get slower
eval = Eval_Fn(model=IRNN, 
               root = root, gamma=gamma, plot=False, n_folds=n_folds, verbose=False)

optimizer = BayesianOptimization(
    f=eval,
    pbounds=model.pbounds,
    random_state=1,
    verbose=2
)

optimizer = load_steps(root, optimizer)

for _ in range(100):
    optimizer.maximize(
        init_points=10,
        n_iter=10)

    save_steps(root, optimizer)